In [2]:

def random_image(h, w, c=3):
    image = torch.randn(c, h, w).mul(0.001).unsqueeze(0)
    return image

def random_image_like(base_image):
    _, C, H, W = base_image.size()
    return random_image(H, W, C)


def optimize(stylenet, img, num_iterations, output_path, original_colors, print_iter, save_iter):
    def iterate():
        t[0] += 1
        optimizer.zero_grad()
        stylenet(img)
        loss = stylenet.get_loss()
        loss.backward()
        maybe_print(stylenet, t[0], print_iter, num_iterations, loss)
        maybe_save(img, t[0], save_iter, num_iterations, original_colors, output_path)
        return loss
    img = nn.Parameter(img)
    optimizer, loopVal = setup_optimizer(img, stylenet.params, num_iterations)
    t = [0]
    while t[0] <= loopVal:
        optimizer.step(iterate)

def load_image(path, image_size, to_normalize=True):
    image = preprocess(path, image_size, to_normalize)
    return image

        
import os
import torch
import torch.nn as nn
from model import *
from utils import *




style_paths = ['examples/inputs/starry_night.jpg','examples/inputs/cubist.jpg']
style_seg_paths = None
style_blend_weights = None
content_path = 'examples/inputs/monalisa.jpg'
content_seg_paths = ['examples/segments/monalisa1a.png','examples/segments/monalisa1b.png']
#content_seg_paths = ['examples/segments/monalisa1b.png','examples/segments/monalisa1a.png']
image_size = 512
content_weight = 5e0
style_weight = 1e2
tv_weight = 1e-3
num_iterations = 500
init = 'random'
init_path = None
output_path = 'out1z.png'
original_colors = 0
style_scale = 1.0
print_iter = 100
save_iter = 100


# setup
params = StylenetArgs()
params.gpu = '0'
params.backend = 'cudnn'

dtype, multidevice, backward_device = setup_gpu(params)
stylenet = StyleNet(params, dtype, multidevice, backward_device)

# load content image
content_image = load_image(content_path, image_size).type(dtype)

# load style images
style_paths = get_style_image_paths(style_paths)
style_size = int(image_size * style_scale)
style_images = [load_image(path, style_size).type(dtype) for path in style_paths]
#style_images = style_images[0]
content_masks = [load_image(path, image_size, to_normalize=False).type(dtype)[0][0] for path in content_seg_paths]

stylenet.capture(content_image, style_images, None, content_masks)#, None, content_masks, style_masks)
#stylenet.capture(content_image, style_images, style_blend_weights, content_masks, style_masks)
            
#img = load_image(init_path, (content_image.size(2), content_image.size(3)), dtype)
img = random_image_like(content_image).type(stylenet.dtype)
optimize(stylenet, img, num_iterations, output_path, original_colors, print_iter, save_iter)


VGG-19 Architecture Detected
Successfully loaded models/vgg19-d01eb7cb.pth
conv1_1: 64 3 3 3
conv1_2: 64 64 3 3
conv2_1: 128 64 3 3
conv2_2: 128 128 3 3
conv3_1: 256 128 3 3
conv3_2: 256 256 3 3
conv3_3: 256 256 3 3
conv3_4: 256 256 3 3
conv4_1: 512 256 3 3
conv4_2: 512 512 3 3
conv4_3: 512 512 3 3
conv4_4: 512 512 3 3
conv5_1: 512 512 3 3
conv5_2: 512 512 3 3
conv5_3: 512 512 3 3
conv5_4: 512 512 3 3
Setting up style layer 2: relu1_1
Setting up style layer 7: relu2_1
Setting up style layer 12: relu3_1
Setting up style layer 21: relu4_1
Setting up content layer 23: relu4_2
Setting up style layer 30: relu5_1
Sequential(
  (0): TVLoss()
  (1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): ReLU(inplace=True)
  (3): MaskedStyleLoss(
    (crit): MSELoss()
  )
  (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): ReLU(inplace=True)
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(64, 128, kerne